# ETL: Silver → Gold

## Contexto do Projeto

Este notebook realiza o processo de ETL (Extract, Transform, Load) para transformar dados da camada Silver para a camada Gold em um pipeline de dados. A camada Gold é destinada a análises avançadas e relatórios, contendo dados limpos, agregados e otimizados para consultas.

## Processo ETL
1. **Extração (Extract)**:
   - Os dados são extraídos da tabela Silver, que contém dados semi-estruturados e pré-processados. 

2. **Transformação (Transform)**:
    - Cria dimensões e fatos a partir dos dados extraídos (Star Schema).

3. **Carga (Load)**:
   - Os dados transformados são carregados no schema Gold.

## 1. Configuração e Imports

In [31]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import psycopg2
from psycopg2.extras import execute_batch
import warnings

warnings.filterwarnings('ignore')

In [32]:
# Configuração de conexão com o banco de dados PostgreSQL
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5433'),
    'database': os.getenv('DB_NAME', 'grupo08'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'postgres')
}

## 2. Extração de Dados da Camada Silver

In [33]:
# Estabelecendo a conexão com o banco de dados
try:
    conn = psycopg2.connect(**DB_CONFIG)
    print("Conexão com o banco de dados estabelecida com sucesso.")
except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")

# Extração de Dados da Camada Silver
silver_table = "silver.filmes"
try:
    df = pd.read_sql_query(f"SELECT * FROM {silver_table};", conn)
    print("Dados extraídos com sucesso.")
    print(f"Número de registros extraídos: {len(df)}")
    print(f"Número de colunas extraídas: {len(df.columns)}")
except Exception as e:
    print(f"Erro ao extrair dados: {e}")

Conexão com o banco de dados estabelecida com sucesso.
Dados extraídos com sucesso.
Número de registros extraídos: 1174587
Número de colunas extraídas: 29


## 3. Transformação dos Dados e Carga na Camada Gold

Nesta etapa, será criado o Star Schema contendo as tabelas de dimensão e fato. Utilizaremos Surrogate Keys (SK) e aplicaremos o padrão mnemônico na nomenclatura das colunas.

In [34]:
# Criando schema DW (Data Warehouse) se não existir
ddl_schema = "CREATE SCHEMA IF NOT EXISTS dw;"

try:
    with conn.cursor() as cursor:
        cursor.execute(ddl_schema)
        conn.commit()
    print("Schema 'dw' verificado/criado com sucesso.")
except Exception as e:
    print(f"Erro ao criar/verificar schema 'dw': {e}")

Schema 'gold' verificado/criado com sucesso.


In [35]:
print ("Criando as tabelas fato e dimensão na camada Gold...")

# 1. DIMENSÃO RELEASE (CALENDÁRIO)
try:
    with conn.cursor() as cur:
        print("Criando dimensão Release...")
        
        sql_dim_release = """
        DROP TABLE IF EXISTS dw.dim_rel CASCADE;

        CREATE TABLE dw.dim_rel AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY release_date) AS srk_rel,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            release_date          AS rel_dat,
            release_year          AS rel_yer,
            release_month         AS rel_mon,
            release_month_name    AS rel_mon_nam,
            release_day_of_week   AS rel_day_wek,
            release_day_name      AS rel_day_nam,
            release_decade        AS rel_dec
            
        FROM (
            -- Calcula campos temporais a partir de release_date
            SELECT DISTINCT
                release_date,
                EXTRACT(YEAR FROM release_date)::INTEGER AS release_year,
                EXTRACT(MONTH FROM release_date)::INTEGER AS release_month,
                TO_CHAR(release_date, 'Month') AS release_month_name,
                EXTRACT(DOW FROM release_date)::INTEGER AS release_day_of_week,
                TO_CHAR(release_date, 'Day') AS release_day_name,
                (EXTRACT(YEAR FROM release_date)::INTEGER / 10 * 10)::INTEGER AS release_decade
            FROM silver.filmes 
            WHERE release_date IS NOT NULL
        ) sub;

        -- Definindo a chave primária física
        ALTER TABLE dw.dim_rel ADD PRIMARY KEY (srk_rel);
        """
        cur.execute(sql_dim_release)
        print("dw.dim_rel criada com sucesso.")

        df_dim_release = pd.read_sql_query("SELECT * FROM dw.dim_rel;", conn)
        print(f"Número de registros em dw.dim_rel: {len(df_dim_release)}")
        print(f"Número de colunas em dw.dim_rel: {len(df_dim_release.columns)}")
        print(df_dim_release.head())
    
except Exception as e:
    print(f"Erro ao criar dw.dim_rel: {e}")

finally:
    conn.commit()

Criando as tabelas fato e dimensão na camada Gold...
Criando dimensão Release...
Erro ao criar dw.dim_rel: schema "dw" does not exist



In [36]:
# 2. DIMENSÃO LUCRO

try:
    with conn.cursor() as cur:
        print("Criando dimensão Profit...")
        
        sql_dim_profit = """
        DROP TABLE IF EXISTS dw.dim_pft CASCADE;

        CREATE TABLE dw.dim_pft AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY budget, revenue) AS srk_pft,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            budget          AS bdg,
            revenue         AS rev,
            profit          AS pft,
            roi             AS ret_inv,
            is_profitable   AS pfe,
            budget_tier     AS bdg_tir

        FROM (
            -- Seleciona combinações únicas de valores financeiros
            SELECT DISTINCT 
                budget, revenue, profit, roi, is_profitable, budget_tier
            FROM silver.filmes
        ) sub;

        ALTER TABLE dw.dim_pft ADD PRIMARY KEY (srk_pft);
        """
        cur.execute(sql_dim_profit)
        print("dw.dim_pft criada com sucesso.")

        df_dim_profit = pd.read_sql_query("SELECT * FROM dw.dim_pft;", conn)
        print(f"Número de registros em dw.dim_pft: {len(df_dim_profit)}")
        print(f"Número de colunas em dw.dim_pft: {len(df_dim_profit.columns)}")
        print(df_dim_profit.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()


Criando dimensão Profit...
Erro na execução: schema "dw" does not exist



In [37]:
# 3. DIMENSÃO ENGAGEMENT

try:
    with conn.cursor() as cur:
        print("Criando dimensão Engagement...")
        
        sql_dim_engagement = """
        DROP TABLE IF EXISTS dw.dim_eng CASCADE;

        CREATE TABLE dw.dim_eng AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY popularity DESC) AS srk_eng,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            vote_average    AS vot_avg,
            vote_count      AS vot_cnt,
            popularity      AS pop

        FROM (
            -- Seleciona combinações únicas de engajamento
            SELECT DISTINCT 
                vote_average, vote_count, popularity
            FROM silver.filmes
        ) sub;

        ALTER TABLE dw.dim_eng ADD PRIMARY KEY (srk_eng);
        """
        cur.execute(sql_dim_engagement)
        print("dw.dim_eng criada com sucesso.")

        df_dim_engagement = pd.read_sql_query("SELECT * FROM dw.dim_eng;", conn)
        print(f"Número de registros em dw.dim_eng: {len(df_dim_engagement)}")
        print(f"Número de colunas em dw.dim_eng: {len(df_dim_engagement.columns)}")
        print(df_dim_engagement.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Engagement...
Erro na execução: schema "dw" does not exist



In [38]:
# 4. DIMENSÃO LANGUAGE

try:
    with conn.cursor() as cur:
        print("Criando dimensão Language...")
        
        sql_dim_language = """
        DROP TABLE IF EXISTS dw.dim_lng CASCADE;

        CREATE TABLE dw.dim_lng AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY original_language) AS srk_lng,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            original_language AS lng

        FROM (
            -- Seleciona línguas únicas
            SELECT DISTINCT original_language
            FROM silver.filmes
            WHERE original_language IS NOT NULL AND original_language <> ''
        ) sub;

        ALTER TABLE dw.dim_lng ADD PRIMARY KEY (srk_lng);
        """
        cur.execute(sql_dim_language)
        print("dw.dim_lng criada com sucesso.")

        df_dim_language = pd.read_sql_query("SELECT * FROM dw.dim_lng;", conn)
        print(f"Número de registros em dw.dim_lng: {len(df_dim_language)}")
        print(f"Número de colunas em dw.dim_lng: {len(df_dim_language.columns)}")
        print(df_dim_language.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Language...
Erro na execução: schema "dw" does not exist



In [39]:
# 5. DIMENSÃO COMPANY

try:
    print ("Criando dimensão Company...")
    with conn.cursor() as cur:
        
        sql_dim_company = """
        DROP TABLE IF EXISTS dw.dim_cmp CASCADE;

        CREATE TABLE dw.dim_cmp AS
        WITH uniao_companies AS (
            SELECT 
                TRIM(UNNEST(STRING_TO_ARRAY(production_companies, ','))) AS nm_company
            FROM silver.filmes
            
            UNION
            
            SELECT 
                TRIM(primary_company) AS nm_company
            FROM silver.filmes
        )
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY nm_company) AS srk_cmp,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            nm_company AS prd_cmp
            
        FROM uniao_companies
        WHERE nm_company IS NOT NULL AND nm_company <> '';

        -- Definindo a chave primária física
        ALTER TABLE dw.dim_cmp ADD PRIMARY KEY (srk_cmp);
        """
        
        cur.execute(sql_dim_company)
        conn.commit()
        print("dw.dim_cmp criada com sucesso.")
        df_dim_company = pd.read_sql_query("SELECT * FROM dw.dim_cmp;", conn)
        print(f"Número de registros em dw.dim_cmp: {len(df_dim_company)}")
        print(f"Número de colunas em dw.dim_cmp: {len(df_dim_company.columns)}")
        print(df_dim_company.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Company...
Erro na execução: schema "dw" does not exist



In [40]:
# 6. DIMENSÃO COUNTRY

try:
    print ("Criando dimensão Country...")
    with conn.cursor() as cur:
        
        sql_dim_country = """
        DROP TABLE IF EXISTS dw.dim_ctr CASCADE;
        CREATE TABLE dw.dim_ctr AS
        WITH uniao_countries AS (
            SELECT 
                TRIM(UNNEST(STRING_TO_ARRAY(production_countries, ','))) AS nm_country
            FROM silver.filmes
            
            UNION
            
            SELECT 
                TRIM(primary_country) AS nm_country
            FROM silver.filmes
        )
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY nm_country) AS srk_ctr,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            nm_country AS prd_ctr
            
        FROM uniao_countries
        WHERE nm_country IS NOT NULL AND nm_country <> '';

        -- Definindo a chave primária física
        ALTER TABLE dw.dim_ctr ADD PRIMARY KEY (srk_ctr);
        """
        
        cur.execute(sql_dim_country)
        conn.commit()
        print("dw.dim_ctr criada com sucesso.")
        df_dim_country = pd.read_sql_query("SELECT * FROM dw.dim_ctr;", conn)
        print(f"Número de registros em dw.dim_ctr: {len(df_dim_country)}")
        print(f"Número de colunas em dw.dim_ctr: {len(df_dim_country.columns)}")
        print(df_dim_country.head())
except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Country...
Erro na execução: schema "dw" does not exist



In [41]:
# 7. DIMENSÃO RUNTIME

try:
    with conn.cursor() as cur:
        print("Criando dimensão Runtime...")
        
        sql_dim_runtime = """
        DROP TABLE IF EXISTS dw.dim_rte CASCADE;
        CREATE TABLE dw.dim_rte AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY runtime) AS srk_rte,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            runtime AS rte

        FROM (
            -- Seleciona runtime únicos
            SELECT DISTINCT runtime
            FROM silver.filmes
            WHERE runtime IS NOT NULL
        ) sub;

        ALTER TABLE dw.dim_rte ADD PRIMARY KEY (srk_rte);
        """
        cur.execute(sql_dim_runtime)
        print("dw.dim_rte criada com sucesso.")

        df_dim_runtime = pd.read_sql_query("SELECT * FROM dw.dim_rte;", conn)
        print(f"Número de registros em dw.dim_rte: {len(df_dim_runtime)}")
        print(f"Número de colunas em dw.dim_rte: {len(df_dim_runtime.columns)}")
        print(df_dim_runtime.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Runtime...
Erro na execução: schema "dw" does not exist



In [42]:
# 8. DIMENSÃO GENRES

try:
    with conn.cursor() as cur:
        print("Criando dimensão Genres...")
        
        sql_dim_genres = """
        DROP TABLE IF EXISTS dw.dim_gen CASCADE;
        CREATE TABLE dw.dim_gen AS
        SELECT
            -- Surrogate Key
            ROW_NUMBER() OVER (ORDER BY genre) AS srk_gen,
            
            -- Atributo de Negócio
            genre AS gen

        FROM (
            -- Explode os gêneros separados por vírgula
            SELECT DISTINCT
                TRIM(genre) AS genre
            FROM silver.filmes,
                 LATERAL unnest(string_to_array(genres, ',')) AS genre
            WHERE genres IS NOT NULL
        ) sub;

        ALTER TABLE dw.dim_gen ADD PRIMARY KEY (srk_gen);
        """
        
        cur.execute(sql_dim_genres)
        print("dw.dim_gen criada com sucesso.")

        df_dim_genres = pd.read_sql_query(
            "SELECT * FROM dw.dim_gen ORDER BY srk_gen;",
            conn
        )

        print(f"Número de registros em dw.dim_gen: {len(df_dim_genres)}")
        print(f"Número de colunas em dw.dim_gen: {len(df_dim_genres.columns)}")
        print(df_dim_genres.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Genres...
Erro na execução: schema "dw" does not exist



In [43]:
# 9. DIMENSÃO PRODUCTION COMPANIES

try:
    with conn.cursor() as cur:
        print("Criando dimensão Production Companies...")
        
        sql_dim_production_companies = """
        DROP TABLE IF EXISTS dw.dim_prd_cmp CASCADE;
        CREATE TABLE dw.dim_prd_cmp AS
        SELECT
            -- Surrogate Key
            ROW_NUMBER() OVER (ORDER BY company) AS srk_prd,
            
            -- Atributo de Negócio
            company AS prd

        FROM (
            -- Explode as produtoras separadas por vírgula
            SELECT DISTINCT
                TRIM(company) AS company
            FROM silver.filmes,
                 LATERAL unnest(string_to_array(production_companies, ',')) AS company
            WHERE production_companies IS NOT NULL
        ) sub;

        ALTER TABLE dw.dim_prd_cmp ADD PRIMARY KEY (srk_prd);
        """
        
        cur.execute(sql_dim_production_companies)
        print("dw.dim_prd_cmp criada com sucesso.")

        df_dim_production_companies = pd.read_sql_query(
            "SELECT * FROM dw.dim_prd_cmp ORDER BY srk_prd;",
            conn
        )

        print(f"Número de registros em dw.dim_prd_cmp: {len(df_dim_production_companies)}")
        print(f"Número de colunas em dw.dim_prd_cmp: {len(df_dim_production_companies.columns)}")
        print(df_dim_production_companies.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Production Companies...
Erro na execução: schema "dw" does not exist



In [44]:
# 10. TABELA PONTE: FILME <-> PRODUTORAS (Relação N:N)

try:
    with conn.cursor() as cur:
        print("Criando tabela ponte Filme-Produtoras...")
        
        sql_bridge_movie_company = """
        DROP TABLE IF EXISTS dw.brg_mov_prd CASCADE;
        
        CREATE TABLE dw.brg_mov_prd AS
        SELECT
            f.id AS movie_id,
            dpc.srk_prd
        FROM silver.filmes f
        CROSS JOIN LATERAL unnest(string_to_array(f.production_companies, ',')) AS company
        INNER JOIN dw.dim_prd_cmp dpc 
            ON TRIM(company) = dpc.prd
        WHERE f.production_companies IS NOT NULL;

        -- Adicionar índices para performance
        CREATE INDEX idx_brg_mov_prd_movie ON dw.brg_mov_prd(movie_id);
        CREATE INDEX idx_brg_mov_prd_prd ON dw.brg_mov_prd(srk_prd);
        
        -- Adicionar chave estrangeira para dimensão
        ALTER TABLE dw.brg_mov_prd 
            ADD CONSTRAINT fk_brg_prd FOREIGN KEY (srk_prd) REFERENCES dw.dim_prd_cmp(srk_prd);
        """
        
        cur.execute(sql_bridge_movie_company)
        print("dw.brg_mov_prd criada com sucesso.")

        df_bridge = pd.read_sql_query(
            "SELECT * FROM dw.brg_mov_prd LIMIT 10;",
            conn
        )

        total_relacoes = pd.read_sql_query("SELECT COUNT(*) FROM dw.brg_mov_prd;", conn).iloc[0,0]
        print(f"Total de relações filme-produtora: {total_relacoes}")
        print(df_bridge.head(10))

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando tabela ponte Filme-Produtoras...
Erro na execução: relation "dw.dim_prd_cmp" does not exist
LINE 10:         INNER JOIN dw.dim_prd_cmp dpc 
                            ^



In [45]:
# 11. FATO FILME

try:
    with conn.cursor() as cur:
        print("Criando tabela fato filme...")
        
        # Etapa 1: Criar a Tabela com os Join
        sql_fact_movie = """
        DROP TABLE IF EXISTS dw.fat_mov CASCADE;

        CREATE TABLE dw.fat_mov AS
        SELECT
            -- Criação da Surrogate Key do Fato (SRK)
            ROW_NUMBER() OVER (ORDER BY f.id) AS srk_ttl,
            
            -- ID original para ligação com tabela ponte
            f.id AS movie_id,
            
            -- Atributos do Filme
            f.title                   AS ttl,
            f.original_title          AS org_ttl,
            f.created_at              AS crt,

            -- Chaves Estrangeiras para Dimensões
            dr.srk_rel,
            dg.srk_gen,
            dp.srk_pft,
            de.srk_eng,
            dl.srk_lng,
            dc.srk_cmp,
            dct.srk_ctr,
            drt.srk_rte

        FROM silver.filmes f
        
        -- 1. Dimensão Release
        LEFT JOIN dw.dim_rel dr 
        ON f.release_date = dr.rel_dat

        -- 2. Dimensão Gênero
        LEFT JOIN dw.dim_gen dg
        ON TRIM(f.primary_genre) = dg.gen

        -- 3. Dimensão Profit
        LEFT JOIN dw.dim_pft dp
        ON f.budget = dp.bdg AND f.revenue = dp.rev AND f.profit = dp.pft
        
        -- 4. Dimensão Engagement
        LEFT JOIN dw.dim_eng de
        ON f.popularity = de.pop AND f.vote_average = de.vot_avg AND f.vote_count = de.vot_cnt
        
        -- 5. Dimensão Language
        LEFT JOIN dw.dim_lng dl
        ON f.original_language = dl.lng

        -- 6. Dimensão Company (Produtora Principal)
        LEFT JOIN dw.dim_cmp dc
        ON TRIM(f.primary_company) = dc.prd_cmp

        -- 7. Dimensão Country
        LEFT JOIN dw.dim_ctr dct
        ON TRIM(f.primary_country) = dct.prd_ctr

        -- 8. Dimensão Runtime
        LEFT JOIN dw.dim_rte drt
        ON f.runtime = drt.rte
        
        WHERE f.title IS NOT NULL;
       """
        cur.execute(sql_fact_movie)
        print("Tabela dw.fat_mov criada (dados inseridos).")

        # Adicionar Chaves Primárias e Estrangeiras
        print("Aplicando chaves e restrições...")
        sql_constraints = """
        ALTER TABLE dw.fat_mov ADD PRIMARY KEY (srk_ttl);

        -- Chave Estrangeira para Dimensões
        ALTER TABLE dw.fat_mov ADD CONSTRAINT frk_rel FOREIGN KEY (srk_rel) REFERENCES dw.dim_rel(srk_rel);
        ALTER TABLE dw.fat_mov ADD CONSTRAINT frk_gen FOREIGN KEY (srk_gen) REFERENCES dw.dim_gen(srk_gen);
        ALTER TABLE dw.fat_mov ADD CONSTRAINT frk_pft FOREIGN KEY (srk_pft) REFERENCES dw.dim_pft(srk_pft);
        ALTER TABLE dw.fat_mov ADD CONSTRAINT frk_eng FOREIGN KEY (srk_eng) REFERENCES dw.dim_eng(srk_eng);
        ALTER TABLE dw.fat_mov ADD CONSTRAINT frk_lng FOREIGN KEY (srk_lng) REFERENCES dw.dim_lng(srk_lng);
        ALTER TABLE dw.fat_mov ADD CONSTRAINT frk_cmp FOREIGN KEY (srk_cmp) REFERENCES dw.dim_cmp(srk_cmp);
        ALTER TABLE dw.fat_mov ADD CONSTRAINT frk_ctr FOREIGN KEY (srk_ctr) REFERENCES dw.dim_ctr(srk_ctr);
        ALTER TABLE dw.fat_mov ADD CONSTRAINT frk_rte FOREIGN KEY (srk_rte) REFERENCES dw.dim_rte(srk_rte);

        """
        cur.execute(sql_constraints)
        print("Chaves aplicadas com sucesso.")

        # Atualizar tabela ponte para usar srk_ttl ao invés de movie_id
        print("Atualizando tabela ponte com srk_ttl...")
        sql_update_bridge = """
        DROP TABLE IF EXISTS dw.brg_mov_prd_temp;
        
        CREATE TABLE dw.brg_mov_prd_temp AS
        SELECT 
            fm.srk_ttl,
            bp.srk_prd
        FROM dw.brg_mov_prd bp
        INNER JOIN dw.fat_mov fm ON bp.movie_id = fm.movie_id;
        
        DROP TABLE dw.brg_mov_prd;
        
        ALTER TABLE dw.brg_mov_prd_temp RENAME TO brg_mov_prd;
        
        -- Recriar índices e constraints
        CREATE INDEX idx_brg_mov_prd_ttl ON dw.brg_mov_prd(srk_ttl);
        CREATE INDEX idx_brg_mov_prd_prd ON dw.brg_mov_prd(srk_prd);
        ALTER TABLE dw.brg_mov_prd ADD CONSTRAINT fk_brg_ttl FOREIGN KEY (srk_ttl) REFERENCES dw.fat_mov(srk_ttl);
        ALTER TABLE dw.brg_mov_prd ADD CONSTRAINT fk_brg_prd FOREIGN KEY (srk_prd) REFERENCES dw.dim_prd_cmp(srk_prd);
        """
        cur.execute(sql_update_bridge)
        print("Tabela ponte atualizada com srk_ttl.")

        print("Criando índices de performance...")
        sql_indexes = """
        CREATE INDEX idx_fat_mov_srk_rel ON dw.fat_mov(srk_rel);
        CREATE INDEX idx_fat_mov_srk_gen ON dw.fat_mov(srk_gen);
        CREATE INDEX idx_fat_mov_srk_pft ON dw.fat_mov(srk_pft);
        CREATE INDEX idx_fat_mov_srk_eng ON dw.fat_mov(srk_eng);
        CREATE INDEX idx_fat_mov_srk_lng ON dw.fat_mov(srk_lng);
        CREATE INDEX idx_fat_mov_srk_cmp ON dw.fat_mov(srk_cmp);
        CREATE INDEX idx_fat_mov_srk_ctr ON dw.fat_mov(srk_ctr);
        CREATE INDEX idx_fat_mov_srk_rte ON dw.fat_mov(srk_rte);
        CREATE INDEX idx_fat_mov_movie_id ON dw.fat_mov(movie_id);
        """
        cur.execute(sql_indexes)
        print("Índices criados com sucesso.")

        # Validação
        df_fact_movie = pd.read_sql_query("SELECT * FROM dw.fat_mov LIMIT 5;", conn)
        print(f"Colunas: {list(df_fact_movie.columns)}")
        print(df_fact_movie.head())
        
        # Validação da tabela ponte
        total_bridge = pd.read_sql_query("SELECT COUNT(*) FROM dw.brg_mov_prd;", conn).iloc[0,0]
        print(f"\nTabela ponte atualizada: {total_bridge} relações filme-produtora")

except Exception as e:
    conn.rollback()
    print(f"Erro na execução da Fato: {e}")

finally:
    conn.commit()

Criando tabela fato filme...
Erro na execução da Fato: relation "dw.dim_rel" does not exist
LINE 30:         LEFT JOIN dw.dim_rel dr 
                           ^



## Validação e Testes

In [46]:
try:
    with conn.cursor() as cur:
        antes = pd.read_sql_query("SELECT COUNT(*) FROM silver.filmes WHERE title IS NOT NULL;", conn).iloc[0,0]
        depois = pd.read_sql_query("SELECT COUNT(*) FROM dw.fat_mov;", conn).iloc[0,0]
        if antes == depois:
            print(f"Validação bem-sucedida: {antes} registros na silver.filmes e {depois} registros na dw.fat_mov.")
        else:
            print(f"Atenção: {antes} registros na silver.filmes, mas {depois} registros na dw.fat_mov.")
except Exception as e:
    print(f"Erro durante a validação: {e}")

Erro durante a validação: Execution failed on sql 'SELECT COUNT(*) FROM dw.fat_mov;': relation "dw.fat_mov" does not exist
LINE 1: SELECT COUNT(*) FROM dw.fat_mov;
                             ^

